In [1]:
import pygame
import random

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Q Learning Agent Settings for learning : 

In [2]:
### Create Q-Learning Table

actions = [0,1]

buckets = 1
Q_table = {}
for i in range(-600, 1, buckets): 
    for x in range(-50, 440, buckets): 
        for j in actions:
            if i<-400: 
                Q_table[((i,x),1)] = 1
                Q_table[((i,x),0)] = 0

            if i>=-400:
                Q_table[((i,x),1)] = 0
                Q_table[((i,x),0)] = 1

def R(y): # Reward function
    rew = 1000 - abs(-380-y)    
    if abs(-380-y) < 50:
        rew = 100  
    if abs(-380-y) >= 50:
        rew = 0
    return rew


def get_bucket(state):
    diffy, diffx = state
    if diffy < -600:
        diffy = -600
    elif diffy > 0:
        diffy = 0
    
    if diffx < -50:
        diffx = -50
    elif diffx > 400:
        diffx = 400

    return buckets*int(diffy/buckets), buckets*int(diffx/buckets)

exploration_rate = 0.001
discount_factor = 0.1


def Action_RL(state):
    if random.random() < exploration_rate:
        return random.choice(actions)
    else:
        return max(actions, key = lambda a: Q_table[get_bucket(state), a])

def Update_RL(state, action, reward, next_state):
    Q_table[get_bucket(state), action] += 0.1*(reward + discount_factor*max(Q_table[get_bucket(next_state), a] for a in actions) - Q_table[get_bucket(state), action])

In [4]:
def launch_game_without_display(verbose=False):


    screen_width = 864
    screen_height = 936
    ground_scroll = 0
    scroll_speed = 4

    score = 0

    distance_between_pipes = 350
    pipe_gape = 200

    flying = True
    game_over = False
    pass_pipe = False

    first_tick = True

    ############################################################
    ################### BIRD ###################################
    ############################################################
    class Bird():
        def __init__(self,x,y):
            self.x = x #center
            self.y = y #center
            self.vel = 0
            

        def getRight(self):
            return self.x + 26
        def getLeft(self):
            return self.x - 25
        def getTop(self):
            return self.y -18
        def getBottom(self):
            return self.y + 18

        def update(self, action=None):

            #gravity
            if flying == True:
                self.vel += 0.5
                if self.vel > 8:
                    self.vel = 8
                if self.getBottom() < 768:
                    self.y += int(self.vel)

            if game_over == False:
                ### JUMP

                if (action == 1 and self.getBottom() > 0):
                    
                    if self.vel > -8:
                        self.vel = -10

                ### STOP JUMP
                
                if (action == 0):
                    pass
                    
                
    bird = Bird(100, screen_height//2)


    class Pipe():
        def __init__(self,x,y):
            self.x = x #x du centre de celui du bas
            self.y = y #y du centre de celui du bas
        
        def PipeBasGetRight(self):
            return self.x + 39
        def PipeBasGetLeft(self):
            return self.x - 39
        def PipeBasGetTop(self):
            return self.y - 280
        def PipeBasGetBottom(self):
            return self.y + 280
        
        def PipeHautGetRight(self):
            return self.x + 39
        def PipeHautGetLeft(self):
            return self.x - 39
        def PipeHautGetTop(self):
            return self.y - 280 - pipe_gape - 560 
        def PipeHautGetBottom(self):
            return self.y - 280 - pipe_gape
        
        def update(self):
            self.x -= scroll_speed
            if self.PipeBasGetRight() < 0:
                pipe_list.remove(self)
                #print('pipe removed')
                #print(len(pipe_list))
        
    pipe_list = []
    pipe_list.append(Pipe(899, 848))


    first_tick = True
    running = True
    while running:


        if first_tick == True:
            action = 1

        bird.update(action)
        #print((bird.x, bird.y), bird.getBottom(), bird.getTop(),  bird.getRight(),bird.getLeft())
        
        #print(pipe_list[0].PipeBasGetRight(), pipe_list[0].PipeBasGetLeft(), pipe_list[0].PipeBasGetTop(), pipe_list[0].PipeBasGetBottom())
        #print(pipe_list[0].PipeHautGetRight(), pipe_list[0].PipeHautGetLeft(), pipe_list[0].PipeHautGetTop(), pipe_list[0].PipeHautGetBottom())
        #print(pipe_list[0].x, pipe_list[0].y)
        #print('---')

        #if the game is not over
        if game_over == False:

            #generate pipes 
            x_last_pipe = pipe_list[-1].x
            if screen_width - x_last_pipe > distance_between_pipes:
                new_pipe = Pipe(903, 848 + random.randint(-200,200))
                pipe_list.append(new_pipe)
            for pipe in pipe_list:
                pipe.update()

            #check if bird hits the ground
            if bird.getBottom() > 768:
                print('hit the ground')
                game_over = True
                flying = False

            #check if bird hits the ceiling
            if bird.getTop() < 0:
                
                print('hit the ceiling')
                game_over = True

            
            #check if bird hits a pipe
            for pipe in pipe_list:
                if bird.getRight() > pipe.PipeBasGetLeft() and bird.getLeft() < pipe.PipeBasGetRight(): #if bird is in the x range of the pipe
                    if bird.getBottom() > pipe.PipeBasGetTop() or bird.getTop() < pipe.PipeHautGetBottom():
                        print('hit a pipe')
                        game_over = True
                        flying = False
                        
                
            #check if bird passed a pipe
            if pipe_list[0].PipeBasGetRight() < bird.x and pass_pipe == False:
                pass_pipe = True
                score += 1
                if verbose:
                    print('score :', score)
            if pipe_list[0].PipeBasGetRight() > bird.x:
                pass_pipe = False
                
            
        #Ask RL agent for the next action
        y_bird = bird.y
        
        x_pipe0_right = pipe_list[0].PipeBasGetRight()
        if x_pipe0_right < bird.x - 30: #if the pipe is behind the bird
            next_pipe_height = pipe_list[1].y - 760 #take the next pipe
            next_pipe_x = pipe_list[1].x
            
        else:
            next_pipe_height = pipe_list[0].y - 760 #if the pipe is in front of the bird
            next_pipe_x = pipe_list[0].x
            
        #print(y_bird)
        state = (next_pipe_height-y_bird, next_pipe_x-bird.x)  #state is the difference of height between the bird and the next pipe
        #print('ybird : ', y_bird, 'next_pipe_height', next_pipe_height, 'delta_y', state)
        if first_tick:
            first_tick = False
            previous_state = state
            previous_action = 1

        else:
            reward = 1

            if game_over:
                reward = -100
            #if verbose:
                #print(reward)
                
            #print(previous_action)
            previous_action = action

            #print(previous_state, state)

            Update_RL(previous_state, previous_action, reward, state)

            action = Action_RL(state)
            previous_state = state

            


        if game_over == True:
            #print('game over')
            running = False


    return score
launch_game_without_display(verbose=True)
            

score : 1
score : 2
score : 3
score : 4
score : 5
score : 6
score : 7
score : 8
score : 9
score : 10
score : 11
score : 12
score : 13
score : 14
score : 15
score : 16
score : 17
score : 18
score : 19
score : 20
score : 21
score : 22
score : 23
score : 24
score : 25
score : 26
score : 27
score : 28
score : 29
score : 30
score : 31
score : 32
score : 33
score : 34
score : 35
score : 36
score : 37
score : 38
score : 39
score : 40
score : 41
score : 42
score : 43
score : 44
score : 45
score : 46
score : 47
score : 48
score : 49
score : 50
score : 51
score : 52
score : 53
score : 54
score : 55
score : 56
score : 57
score : 58
score : 59
score : 60
score : 61
score : 62
score : 63
score : 64
score : 65
score : 66
score : 67
score : 68
score : 69
score : 70
score : 71
score : 72
score : 73
score : 74
score : 75
score : 76
score : 77
score : 78
score : 79
score : 80
score : 81
score : 82
hit a pipe


82

# Train the Agent 

In [6]:
max_score = 0
num_episodes = 10000
for i in range(num_episodes):
    score =  launch_game_without_display()
    if score > 0:

        max_score = max(max_score, score)
        print('episode :', i, 'score :', score)

hit a pipe
episode : 0 score : 51
hit a pipe
episode : 1 score : 7
hit a pipe
episode : 2 score : 18
hit a pipe
episode : 3 score : 29
hit a pipe
episode : 4 score : 98
hit a pipe
episode : 5 score : 16
hit a pipe
episode : 6 score : 25
hit a pipe
episode : 7 score : 10
hit a pipe
episode : 8 score : 101
hit a pipe
episode : 9 score : 63
hit a pipe
episode : 10 score : 6
hit a pipe
episode : 11 score : 15
hit a pipe
episode : 12 score : 51
hit a pipe
episode : 13 score : 56
hit a pipe
episode : 14 score : 57
hit a pipe
episode : 15 score : 11
hit a pipe
episode : 16 score : 6
hit a pipe
episode : 17 score : 27
hit a pipe
episode : 18 score : 6
hit a pipe
episode : 19 score : 11
hit a pipe
episode : 20 score : 32
hit a pipe
episode : 21 score : 189
hit a pipe
episode : 22 score : 65
hit a pipe
episode : 23 score : 8
hit a pipe
episode : 24 score : 56
hit a pipe
episode : 25 score : 15
hit a pipe
episode : 26 score : 35
hit a pipe
episode : 27 score : 90
hit a pipe
episode : 28 score : 2

In [ ]:
#save Q table 
import pickle
with open('New Trained Q_table.pkl', 'wb') as f:
    pickle.dump(Q_table, f)

# Test the trained agent with display

- Set the discount factor and the exploration rate to 0 

In [7]:
discount_factor = 0.0
exploration_rate = 0.000

def Update_RL(state, action, reward, next_state):
    pass

In [12]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Constants
screen_width = 864
screen_height = 936
ground_scroll = 0
scroll_speed = 4
pipe_gape = 200

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)


# Create the screen
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Flappy Bird with RL Agent')

# Load images
bird_img = pygame.image.load('./img/bird1.png')
pipe_img = pygame.image.load('./img/pipe_gray.png')
ground_img = pygame.image.load('./img/ground_lunaire.png')
background_img = pygame.image.load('./img/bg.jpg')

# Bird class
class Bird():
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.vel = 0

    def getRight(self):
        return self.x + 26

    def getLeft(self):
        return self.x - 25

    def getTop(self):
        return self.y - 18

    def getBottom(self):
        return self.y + 18

    def update(self, action=None):
        global flying, game_over

        if flying:
            self.vel += 0.5
            if self.vel > 8:
                self.vel = 8
            if self.getBottom() < 768:
                self.y += int(self.vel)

        if not game_over:
            if action == 1 and self.getBottom() > 0:
                if self.vel > -8:
                    self.vel = -10

            if action == 0:
                pass

    def draw(self):
        screen.blit(bird_img, (self.x - 26, self.y - 18))

# Pipe class
class Pipe():
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def PipeBasGetRight(self):
        return self.x + 39

    def PipeBasGetLeft(self):
        return self.x - 39

    def PipeBasGetTop(self):
        return self.y - 280

    def PipeBasGetBottom(self):
        return self.y + 280

    def PipeHautGetRight(self):
        return self.x + 39

    def PipeHautGetLeft(self):
        return self.x - 39

    def PipeHautGetTop(self):
        return self.y - 280 - pipe_gape - 560

    def PipeHautGetBottom(self):
        return self.y - 280 - pipe_gape

    def update(self):
        self.x -= scroll_speed
        if self.PipeBasGetRight() < 0:
            pipe_list.remove(self)

    def draw(self):
        screen.blit(pipe_img, (self.x - 39, self.y - 280))
        screen.blit(pygame.transform.flip(pipe_img, False, True), (self.x - 39, self.y - 280 - pipe_gape - 560))

pipe_list = [Pipe(899, 848)]

def launch_game(verbose=False):
    global flying, game_over

    score = 0
    distance_between_pipes = 350
    pipe_gape = 200

    flying = True
    game_over = False
    pass_pipe = False

    first_tick = True

    bird = Bird(100, screen_height // 2)
    

    running = True
    while running:
        screen.blit(background_img, (0, 0))

        if first_tick:
            action = 1

        bird.update(action)
        bird.draw()

        if not game_over:
            x_last_pipe = pipe_list[-1].x
            if screen_width - x_last_pipe > distance_between_pipes:
                new_pipe = Pipe(903, 848 + random.randint(-200, 200))
                pipe_list.append(new_pipe)

            for pipe in pipe_list:
                pipe.update()
                pipe.draw()

            if bird.getBottom() > 768:
                print('hit the ground')
                game_over = True
                flying = False

            if bird.getTop() < 0:
                print('hit the ceiling')
                game_over = True

            for pipe in pipe_list:
                if bird.getRight() > pipe.PipeBasGetLeft() and bird.getLeft() < pipe.PipeBasGetRight():
                    if bird.getBottom() > pipe.PipeBasGetTop() or bird.getTop() < pipe.PipeHautGetBottom():
                        print('hit a pipe')
                        game_over = True
                        flying = False

            if pipe_list[0].PipeBasGetRight() < bird.x and not pass_pipe:
                pass_pipe = True
                score += 1
                if verbose:
                    print('score :', score)
            if pipe_list[0].PipeBasGetRight() > bird.x:
                pass_pipe = False

        y_bird = bird.y

        x_pipe0_right = pipe_list[0].PipeBasGetRight()
        if x_pipe0_right < bird.x - 30:
            next_pipe_height = pipe_list[1].y - 760
            diffx = pipe_list[1].x - bird.x
        else:
            next_pipe_height = pipe_list[0].y - 760
            diffx = pipe_list[0].x - bird.x
            

        state = (next_pipe_height - y_bird, diffx)
        if first_tick:
            first_tick = False
            previous_state = state
            previous_action = 1
        else:
            reward = 1
            if game_over:
                reward = -100

            previous_action = action
            Update_RL(previous_state, previous_action, reward, state)
            action = Action_RL(state)
            previous_state = state

        if game_over:
            running = False

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        pygame.display.update()

    return score

launch_game(verbose=True)
pygame.quit()


score : 1
score : 2
score : 3
score : 4
score : 5
score : 6
score : 7
score : 8
score : 9
score : 10
score : 11
score : 12
score : 13
score : 14
score : 15
score : 16
score : 17
score : 18
score : 19
score : 20
score : 21
score : 22
score : 23
score : 24
score : 25
score : 26
score : 27
score : 28
score : 29
score : 30
score : 31
score : 32
score : 33
score : 34
score : 35
score : 36
score : 37
score : 38
score : 39
score : 40
score : 41
score : 42
score : 43
score : 44
score : 45
score : 46
score : 47
score : 48
score : 49
score : 50
score : 51
score : 52
score : 53
score : 54
score : 55
score : 56
score : 57
score : 58
score : 59
score : 60
score : 61
score : 62
score : 63
score : 64
score : 65
score : 66
score : 67
score : 68
score : 69
score : 70
score : 71
score : 72
score : 73
score : 74
score : 75
score : 76
score : 77
score : 78
score : 79
score : 80
score : 81
score : 82
score : 83
score : 84
score : 85
score : 86
score : 87
score : 88
score : 89
score : 90
score : 91
score : 

In [13]:
pygame.quit()